### Movenet test 

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed 
import numpy as np
import cv2
import os
import keras
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold


# Matplotlib 
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Animation libraries 
import imageio
from IPython.display import HTML, display

import os
import numpy as np
from PIL import Image

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout

### Load movenet model from TF Hub

In [15]:
model_name = "movenet_thunder"

if "tflite" in model_name:
  if "movenet_lightning_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
    input_size = 256
  elif "movenet_lightning_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/int8/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/int8/4?lite-format=tflite
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_path="model.tflite")
  interpreter.allocate_tensors()

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    # TF Lite format expects tensor type of uint8.
    input_image = tf.cast(input_image, dtype=tf.uint8)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    # Invoke inference.
    interpreter.invoke()
    # Get the model prediction.
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    return keypoints_with_scores

else:
  if "movenet_lightning" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
    input_size = 192
  elif "movenet_thunder" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    model = module.signatures['serving_default']

    # SavedModel format expects tensor type of int32.
    input_image = tf.cast(input_image, dtype=tf.int32)
    # Run model inference.
    outputs = model(input_image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints_with_scores = outputs['output_0'].numpy()
    return keypoints_with_scores

### Load Images from Folders 

In [16]:
def load_all_images():
    # Root directory containing all numbered folders
    root_dir = os.path.join(os.getcwd(), 'TestImages', 'ResizedImages')
    
    # Number of folders (0-15)
    num_folders = 16
    
    # Create lists to store all images and their labels
    all_images = []
    all_labels = []
    
    # Loop through each folder
    for folder_num in range(num_folders):
        folder_path = os.path.join(root_dir, str(folder_num))
        
        # Check if folder exists
        if not os.path.exists(folder_path):
            print(f"Warning: Folder {folder_path} does not exist.")
            continue
            
        # Get all image files from the folder
        image_files = [f for f in os.listdir(folder_path)
                    if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
        
        print(f"Loading {len(image_files)} images from class {folder_num}")
        
        # Load each image
        for img_file in image_files:
            img_path = os.path.join(folder_path, img_file)
            try:
                # Open the image
                img = Image.open(img_path)
                
                # Convert to numpy array
                img_array = np.array(img)
                
                # Add to the lists
                all_images.append(img_array)
                all_labels.append(folder_num)  # Use folder number as label
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    
    # Convert lists to numpy arrays
    X = np.array(all_images)
    y = np.array(all_labels)
    
    print(f"Total images loaded: {len(all_images)}")
    print(f"Images shape: {X.shape}")
    print(f"Labels shape: {y.shape}")
    
    return X, y  # Return both images and labels

# Load all images and labels
images, labels = load_all_images()

Loading 24 images from class 0
Loading 36 images from class 1
Loading 12 images from class 2
Loading 36 images from class 3
Loading 12 images from class 4
Loading 24 images from class 5
Loading 24 images from class 6
Loading 24 images from class 7
Loading 36 images from class 8
Loading 12 images from class 9
Loading 36 images from class 10
Loading 12 images from class 11
Loading 24 images from class 12
Loading 24 images from class 13
Loading 36 images from class 14
Loading 12 images from class 15
Total images loaded: 384
Images shape: (384, 1000, 660, 3)
Labels shape: (384,)


### Predict Image 

In [17]:
# Extract the first image and label
test_image = images[0]
test_label = labels[0]

# Remove the first image
train_images = images[1:]
train_labels = labels[1:]

### Apply Movenet to images to get the key points 

In [18]:
# Resize and pad the image to keep the aspect ratio and fit the expected size.

def keypoints(images):
  keypoints_with_scores = []

  for image in images:
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

    keypoints_with_scores.append(movenet(input_image))


  return keypoints_with_scores


movenet_keypoints = keypoints(images=images)

movenet_keypoints



[array([[[[0.18560174, 0.499188  , 0.6125121 ],
          [0.1477361 , 0.53244126, 0.7594675 ],
          [0.15092541, 0.46471456, 0.75099343],
          [0.17030618, 0.58432966, 0.77345943],
          [0.17060052, 0.42083392, 0.6046614 ],
          [0.32541397, 0.6742303 , 0.7651241 ],
          [0.3646839 , 0.35140046, 0.8484663 ],
          [0.6197824 , 0.7753371 , 0.9108351 ],
          [0.6692008 , 0.29682276, 0.9077188 ],
          [0.48662984, 0.671418  , 0.8171445 ],
          [0.5092567 , 0.39840382, 0.5932305 ],
          [0.8562911 , 0.63918155, 0.51624453],
          [0.8688652 , 0.42692658, 0.51500905],
          [0.98397845, 0.680927  , 0.08208431],
          [0.97975296, 0.37597036, 0.02569294],
          [1.0575154 , 0.37539726, 0.01129639],
          [1.0082326 , 0.5863816 , 0.01488248]]]], dtype=float32),
 array([[[[0.1565897 , 0.49252456, 0.5859899 ],
          [0.12644143, 0.51460886, 0.39750624],
          [0.1306628 , 0.46311986, 0.8445015 ],
          [0.1292333 

### Convert to numPy array and remove unnessasary dimensions

In [8]:
keypoints_array = np.array(movenet_keypoints)
keypoints_array = keypoints_array.squeeze(axis=(1,2))

keypoints_array.shape


(384, 17, 3)

### One hot encoded feature Data 

In [9]:
label_data_hot = keras.utils.to_categorical(labels, num_classes=16)

label_data_hot

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

### Neural Networks - Multilayered 

In [10]:
X_train, X_test, y_train, y_test = train_test_split(keypoints_array, label_data_hot, test_size=0.2, random_state=42)

model = Sequential()
model.add(Flatten(input_shape = (17,3)))
model.add(Dense(units=64, activation='relu'))

model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.2)) 
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.3)) 
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(units=2048, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=label_data_hot.shape[1], activation='softmax'))

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=200, batch_size=35)

Epoch 1/200


/Users/carlkaziboni/anaconda3/envs/balldetection/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.1040 - loss: 2.7657
Epoch 2/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0970 - loss: 2.6741
Epoch 3/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1704 - loss: 2.4633
Epoch 4/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2212 - loss: 2.2450
Epoch 5/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3270 - loss: 2.0703
Epoch 6/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4316 - loss: 1.7594
Epoch 7/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3861 - loss: 1.7056
Epoch 8/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4935 - loss: 1.4624
Epoch 9/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4081 - loss: 1.5366
Epoch 10/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4169 - loss: 1.5741
Epoch 11/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4476 - loss: 1.4073
Epoch 12/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4802 - loss: 1.3447
Epoc

In [11]:
loss, accuracy = model.evaluate(X_test, y_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6554 - loss: 1.4249 


### Image Prediction

In [12]:
temp = tf.expand_dims(test_image, axis=0)
temp = tf.image.resize_with_pad(temp, input_size, input_size)
predict_keypoints = movenet(temp)

predict_keypoints = np.array(predict_keypoints)
predict_keypoints = predict_keypoints[0, 0]  # Now has shape (17, 3)

predict_keypoints_batch = np.expand_dims(predict_keypoints, axis=0)
model.predict(predict_keypoints_batch)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


array([[9.9999964e-01, 2.1589553e-26, 2.1343657e-25, 4.9561126e-19,
        4.7682138e-14, 1.7154559e-25, 1.0106374e-12, 3.1966749e-07,
        2.6437916e-21, 2.6673686e-18, 5.8469530e-18, 1.5051763e-12,
        1.6929175e-10, 1.3544717e-18, 2.7843287e-20, 9.2207545e-25]],
      dtype=float32)

### k folds

In [ ]:
# Split the KFold indexes into separate training and testing datasets 
def KFolds(keypoints_array, label_data_hot, n_splits=5):
    # Initialize KFold
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Initialize lists to store results
    losses = []
    accuracies = []
    
    # Track fold number
    fold_no = 1
    
    # Get the training and testing indexes based on the fold
    for train_index, test_index in kf.split(keypoints_array):
        print(f'Training for fold {fold_no} ...')
        
        # Split the data using the fold indices
        X_train = keypoints_array[train_index]
        y_train = label_data_hot[train_index]
        X_test = keypoints_array[test_index]
        y_test = label_data_hot[test_index]

        # Create and train model for this fold
        model = Sequential()
        model.add(Flatten(input_shape = (17,3)))
        model.add(Dense(units=64, activation='relu'))
        model.add(Dense(units=128, activation='relu'))
        model.add(Dropout(0.2)) 
        model.add(Dense(units=256, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(units=512, activation='relu'))
        model.add(Dropout(0.3)) 
        model.add(Dense(units=1024, activation='relu'))
        model.add(Dropout(0.4))
        model.add(Dense(units=2048, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(units=label_data_hot.shape[1], activation='softmax'))
        
        model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
        
        # Fit the model
        history = model.fit(
            X_train, 
            y_train, 
            epochs=200, 
            batch_size=35, 
            verbose=1
        )
        
        # Evaluate on test data
        loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
        print(f'Fold {fold_no} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')
        
        # Store the metrics for this fold
        losses.append(loss)
        accuracies.append(accuracy)
        
        # Increment fold counter
        fold_no += 1
    
    # Calculate and print average metrics
    avg_loss = sum(losses) / len(losses)
    avg_accuracy = sum(accuracies) / len(accuracies)
    
    print("\n---------- K-Fold Cross Validation Results ----------")
    print(f"Average Loss: {avg_loss:.4f}")
    print(f"Average Accuracy: {avg_accuracy:.4f}")
    print(f"Individual fold accuracies: {[f'{acc:.4f}' for acc in accuracies]}")
    print("----------------------------------------------------")
    
    return losses, accuracies

# Usage
losses, accuracies = KFolds(keypoints_array=keypoints_array, label_data_hot=label_data_hot)

Training for fold 1 ...
Epoch 1/200


c:\Users\Georg\anaconda3\envs\steam_analysis\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.0621 - loss: 2.7538
Epoch 2/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1717 - loss: 2.6582
Epoch 3/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2317 - loss: 2.4492
Epoch 4/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2528 - loss: 2.2356
Epoch 5/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2890 - loss: 2.0591
Epoch 6/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2880 - loss: 1.8929
Epoch 7/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3705 - loss: 1.7415
Epoch 8/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3595 - loss: 1.5980
Epoch 9/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4199 - loss: 1.4574
Epoch 10/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4072 - loss: 1.4536
Epoch 11/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5107 - loss: 1.3302
Epoch 12/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4571 - loss: 1.3750
E

In [21]:
root_dir = os.path.join(os.getcwd(), "MoveNetSmart", "ResizedImages", "Ball")

num_folders = 3
    
# Create lists to store all images and their labels
all_images = []
all_labels = []

# Loop through each folder
for folder_num in range(num_folders):
    folder_path = os.path.join(root_dir, str(folder_num))
    
    # Check if folder exists
    if not os.path.exists(folder_path):
        print(f"Warning: Folder {folder_path} does not exist.")
        continue
        
    # Get all image files from the folder
    image_files = [f for f in os.listdir(folder_path)
                if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
    
    print(f"Loading {len(image_files)} images from class {folder_num}")
    
    # Load each image
    for img_file in image_files:
        img_path = os.path.join(folder_path, img_file)
        try:
            # Open the image
            img = Image.open(img_path)
            
            # Convert to numpy array
            img_array = np.array(img)
            
            # Add to the lists
            all_images.append(img_array)
            all_labels.append(folder_num)  # Use folder number as label
        except Exception as e:
            print(f"Error loading {img_path}: {e}")

# Convert lists to numpy arrays
X = np.array(all_images)
y = np.array(all_labels)

print(f"Total images loaded: {len(all_images)}")
print(f"Images shape: {X.shape}")
print(f"Labels shape: {y.shape}")

images, labels = (X, y)

Loading 24 images from class 0
Loading 36 images from class 1
Loading 12 images from class 2
Total images loaded: 72
Images shape: (72, 1000, 660, 3)
Labels shape: (72,)


In [23]:
# Resize and pad the image to keep the aspect ratio and fit the expected size.

def keypoints(images):
  keypoints_with_scores = []

  for image in images:
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

    keypoints_with_scores.append(movenet(input_image))


  return keypoints_with_scores


movenet_keypoints = keypoints(images=images)

movenet_keypoints

[array([[[[0.09301417, 0.51009536, 0.6494443 ],
          [0.07387364, 0.5271524 , 0.7787871 ],
          [0.07530379, 0.49236032, 0.785872  ],
          [0.08465802, 0.551636  , 0.90527576],
          [0.08897706, 0.47008264, 0.7637128 ],
          [0.19849536, 0.59941983, 0.9367317 ],
          [0.19556278, 0.42292976, 0.8625939 ],
          [0.3000966 , 0.6710868 , 0.9089519 ],
          [0.29493916, 0.3268115 , 0.7394176 ],
          [0.39452875, 0.7030987 , 0.70636463],
          [0.41013172, 0.38403353, 0.8663399 ],
          [0.48446637, 0.56238896, 0.8674178 ],
          [0.47930023, 0.4509552 , 0.7810429 ],
          [0.6887402 , 0.581436  , 0.73265296],
          [0.68427706, 0.42484346, 0.69896775],
          [0.89571935, 0.5900788 , 0.8690883 ],
          [0.8950411 , 0.4041374 , 0.8016925 ]]]], dtype=float32),
 array([[[[0.1209837 , 0.49950674, 0.43631586],
          [0.10422722, 0.5176335 , 0.7102399 ],
          [0.10351356, 0.4829475 , 0.60308486],
          [0.11136356

In [24]:
keypoints_array = np.array(movenet_keypoints)
keypoints_array = keypoints_array.squeeze(axis=(1,2))

keypoints_array.shape

(72, 17, 3)

In [ ]:
label_data_hot = keras.utils.to_categorical(labels, num_classes=3)

label_data_hot.shape

(72, 3)

In [29]:
# Split the KFold indexes into separate training and testing datasets 
def KFolds(keypoints_array, label_data_hot, n_splits=5):
    # Initialize KFold
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Initialize lists to store results
    losses = []
    accuracies = []
    
    # Track fold number
    fold_no = 1
    
    # Get the training and testing indexes based on the fold
    for train_index, test_index in kf.split(keypoints_array):
        print(f'Training for fold {fold_no} ...')
        
        # Split the data using the fold indices
        X_train = keypoints_array[train_index]
        y_train = label_data_hot[train_index]
        X_test = keypoints_array[test_index]
        y_test = label_data_hot[test_index]

        # Create and train model for this fold
        model = Sequential()
        model.add(Flatten(input_shape = (17,3)))
        model.add(Dense(units=64, activation='relu'))
        model.add(Dense(units=128, activation='relu'))
        model.add(Dropout(0.2)) 
        model.add(Dense(units=256, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(units=512, activation='relu'))
        model.add(Dropout(0.3)) 
        model.add(Dense(units=1024, activation='relu'))
        model.add(Dropout(0.4))
        model.add(Dense(units=2048, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(units=label_data_hot.shape[1], activation='softmax'))
        
        model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
        
        # Fit the model
        history = model.fit(
            X_train, 
            y_train, 
            epochs=200, 
            batch_size=35, 
            verbose=1
        )
        
        # Evaluate on test data
        loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
        print(f'Fold {fold_no} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')
        
        # Store the metrics for this fold
        losses.append(loss)
        accuracies.append(accuracy)
        
        # Increment fold counter
        fold_no += 1
    
    # Calculate and print average metrics
    avg_loss = sum(losses) / len(losses)
    avg_accuracy = sum(accuracies) / len(accuracies)
    
    print("\n---------- K-Fold Cross Validation Results ----------")
    print(f"Average Loss: {avg_loss:.4f}")
    print(f"Average Accuracy: {avg_accuracy:.4f}")
    print(f"Individual fold accuracies: {[f'{acc:.4f}' for acc in accuracies]}")
    print("----------------------------------------------------")
    
    return losses, accuracies

# Usage
losses, accuracies = KFolds(keypoints_array=keypoints_array, label_data_hot=label_data_hot)

Training for fold 1 ...
Epoch 1/200


/Users/carlkaziboni/anaconda3/envs/balldetection/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.4045 - loss: 1.0881 
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5106 - loss: 1.0293
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5106 - loss: 1.0544
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4916 - loss: 1.0390
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5011 - loss: 1.0077
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5011 - loss: 1.0457
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5011 - loss: 1.0312
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5011 - loss: 1.0520
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5011 - loss: 1.0029
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5318 - loss: 0.9781
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5743 - loss: 1.0334
Epoch 12/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5574 - loss: 1.0211


In [30]:
root_dir = os.path.join(os.getcwd(), "MoveNetSmart", "ResizedImages", "FullBody")

num_folders = 10
    
# Create lists to store all images and their labels
all_images = []
all_labels = []

# Loop through each folder
for folder_num in range(num_folders):
    folder_path = os.path.join(root_dir, str(folder_num))
    
    # Check if folder exists
    if not os.path.exists(folder_path):
        print(f"Warning: Folder {folder_path} does not exist.")
        continue
        
    # Get all image files from the folder
    image_files = [f for f in os.listdir(folder_path)
                if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
    
    print(f"Loading {len(image_files)} images from class {folder_num}")
    
    # Load each image
    for img_file in image_files:
        img_path = os.path.join(folder_path, img_file)
        try:
            # Open the image
            img = Image.open(img_path)
            
            # Convert to numpy array
            img_array = np.array(img)
            
            # Add to the lists
            all_images.append(img_array)
            all_labels.append(folder_num)  # Use folder number as label
        except Exception as e:
            print(f"Error loading {img_path}: {e}")

# Convert lists to numpy arrays
X = np.array(all_images)
y = np.array(all_labels)

print(f"Total images loaded: {len(all_images)}")
print(f"Images shape: {X.shape}")
print(f"Labels shape: {y.shape}")

images, labels = (X, y)

Loading 36 images from class 0
Loading 12 images from class 1
Loading 36 images from class 2
Loading 12 images from class 3
Loading 24 images from class 4
Loading 24 images from class 5
Loading 36 images from class 6
Loading 12 images from class 7
Loading 36 images from class 8
Loading 12 images from class 9
Total images loaded: 240
Images shape: (240, 1000, 660, 3)
Labels shape: (240,)


In [31]:
# Resize and pad the image to keep the aspect ratio and fit the expected size.

def keypoints(images):
  keypoints_with_scores = []

  for image in images:
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

    keypoints_with_scores.append(movenet(input_image))


  return keypoints_with_scores


movenet_keypoints = keypoints(images=images)

movenet_keypoints

[array([[[[0.09492617, 0.4955298 , 0.60966235],
          [0.07642668, 0.51408374, 0.3152466 ],
          [0.075551  , 0.479899  , 0.46473786],
          [0.08767977, 0.5473676 , 0.71281236],
          [0.08884233, 0.4625739 , 0.8723163 ],
          [0.18329981, 0.5848206 , 0.71052325],
          [0.19127432, 0.43264887, 0.8501858 ],
          [0.32486665, 0.5209501 , 0.60306   ],
          [0.3376223 , 0.38686487, 0.6188231 ],
          [0.32219502, 0.4090058 , 0.49599612],
          [0.33096826, 0.4417203 , 0.14953853],
          [0.47049823, 0.543791  , 0.70426893],
          [0.466187  , 0.45539746, 0.6854034 ],
          [0.69475484, 0.56212246, 0.75531894],
          [0.69006455, 0.4721126 , 0.8131054 ],
          [0.921539  , 0.5904425 , 0.7658029 ],
          [0.88669705, 0.4744707 , 0.7421043 ]]]], dtype=float32),
 array([[[[0.11353622, 0.49850297, 0.6614945 ],
          [0.09051926, 0.5170513 , 0.67585105],
          [0.09476189, 0.48517677, 0.523542  ],
          [0.09599116

In [32]:
keypoints_array = np.array(movenet_keypoints)
keypoints_array = keypoints_array.squeeze(axis=(1,2))

keypoints_array.shape

(240, 17, 3)

In [34]:
label_data_hot = keras.utils.to_categorical(labels, num_classes=10)

label_data_hot.shape

(240, 10)

In [35]:
# Split the KFold indexes into separate training and testing datasets 
def KFolds(keypoints_array, label_data_hot, n_splits=5):
    # Initialize KFold
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Initialize lists to store results
    losses = []
    accuracies = []
    
    # Track fold number
    fold_no = 1
    
    # Get the training and testing indexes based on the fold
    for train_index, test_index in kf.split(keypoints_array):
        print(f'Training for fold {fold_no} ...')
        
        # Split the data using the fold indices
        X_train = keypoints_array[train_index]
        y_train = label_data_hot[train_index]
        X_test = keypoints_array[test_index]
        y_test = label_data_hot[test_index]

        # Create and train model for this fold
        model = Sequential()
        model.add(Flatten(input_shape = (17,3)))
        model.add(Dense(units=64, activation='relu'))
        model.add(Dense(units=128, activation='relu'))
        model.add(Dropout(0.2)) 
        model.add(Dense(units=256, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(units=512, activation='relu'))
        model.add(Dropout(0.3)) 
        model.add(Dense(units=1024, activation='relu'))
        model.add(Dropout(0.4))
        model.add(Dense(units=2048, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(units=label_data_hot.shape[1], activation='softmax'))
        
        model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
        
        # Fit the model
        history = model.fit(
            X_train, 
            y_train, 
            epochs=200, 
            batch_size=35, 
            verbose=1
        )
        
        # Evaluate on test data
        loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
        print(f'Fold {fold_no} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')
        
        # Store the metrics for this fold
        losses.append(loss)
        accuracies.append(accuracy)
        
        # Increment fold counter
        fold_no += 1
    
    # Calculate and print average metrics
    avg_loss = sum(losses) / len(losses)
    avg_accuracy = sum(accuracies) / len(accuracies)
    
    print("\n---------- K-Fold Cross Validation Results ----------")
    print(f"Average Loss: {avg_loss:.4f}")
    print(f"Average Accuracy: {avg_accuracy:.4f}")
    print(f"Individual fold accuracies: {[f'{acc:.4f}' for acc in accuracies]}")
    print("----------------------------------------------------")
    
    return losses, accuracies

# Usage
losses, accuracies = KFolds(keypoints_array=keypoints_array, label_data_hot=label_data_hot)

Training for fold 1 ...
Epoch 1/200


/Users/carlkaziboni/anaconda3/envs/balldetection/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.1315 - loss: 2.2962
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1220 - loss: 2.1788
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1981 - loss: 2.1973
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1805 - loss: 2.1658
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1719 - loss: 2.1362 
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1845 - loss: 2.1484 
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3223 - loss: 1.9947 
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3773 - loss: 1.8461
Epoch 9/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3744 - loss: 1.6436 
Epoch 10/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4331 - loss: 1.4648 
Epoch 11/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4726 - loss: 1.3144 
Epoch 12/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4930 - loss: 1.1990
E

In [37]:
root_dir = os.path.join(os.getcwd(), "MoveNetSmart", "ResizedImages", "HalfBody")

num_folders = 2
    
# Create lists to store all images and their labels
all_images = []
all_labels = []

# Loop through each folder
for folder_num in range(num_folders):
    folder_path = os.path.join(root_dir, str(folder_num))
    
    # Check if folder exists
    if not os.path.exists(folder_path):
        print(f"Warning: Folder {folder_path} does not exist.")
        continue
        
    # Get all image files from the folder
    image_files = [f for f in os.listdir(folder_path)
                if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
    
    print(f"Loading {len(image_files)} images from class {folder_num}")
    
    # Load each image
    for img_file in image_files:
        img_path = os.path.join(folder_path, img_file)
        try:
            # Open the image
            img = Image.open(img_path)
            
            # Convert to numpy array
            img_array = np.array(img)
            
            # Add to the lists
            all_images.append(img_array)
            all_labels.append(folder_num)  # Use folder number as label
        except Exception as e:
            print(f"Error loading {img_path}: {e}")

# Convert lists to numpy arrays
X = np.array(all_images)
y = np.array(all_labels)

print(f"Total images loaded: {len(all_images)}")
print(f"Images shape: {X.shape}")
print(f"Labels shape: {y.shape}")

images, labels = (X, y)

Loading 24 images from class 0
Loading 24 images from class 1
Total images loaded: 48
Images shape: (48, 1000, 660, 3)
Labels shape: (48,)


In [38]:
def keypoints(images):
  keypoints_with_scores = []

  for image in images:
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

    keypoints_with_scores.append(movenet(input_image))


  return keypoints_with_scores


movenet_keypoints = keypoints(images=images)

movenet_keypoints

[array([[[[0.18560174, 0.499188  , 0.6125121 ],
          [0.1477361 , 0.53244126, 0.7594675 ],
          [0.15092541, 0.46471456, 0.75099343],
          [0.17030618, 0.58432966, 0.77345943],
          [0.17060052, 0.42083392, 0.6046614 ],
          [0.32541397, 0.6742303 , 0.7651241 ],
          [0.3646839 , 0.35140046, 0.8484663 ],
          [0.6197824 , 0.7753371 , 0.9108351 ],
          [0.6692008 , 0.29682276, 0.9077188 ],
          [0.48662984, 0.671418  , 0.8171445 ],
          [0.5092567 , 0.39840382, 0.5932305 ],
          [0.8562911 , 0.63918155, 0.51624453],
          [0.8688652 , 0.42692658, 0.51500905],
          [0.98397845, 0.680927  , 0.08208431],
          [0.97975296, 0.37597036, 0.02569294],
          [1.0575154 , 0.37539726, 0.01129639],
          [1.0082326 , 0.5863816 , 0.01488248]]]], dtype=float32),
 array([[[[0.1565897 , 0.49252456, 0.5859899 ],
          [0.12644143, 0.51460886, 0.39750624],
          [0.1306628 , 0.46311986, 0.8445015 ],
          [0.1292333 

In [39]:
keypoints_array = np.array(movenet_keypoints)
keypoints_array = keypoints_array.squeeze(axis=(1,2))

keypoints_array.shape

(48, 17, 3)

In [40]:
label_data_hot = keras.utils.to_categorical(labels, num_classes=10)

label_data_hot.shape

(48, 10)

In [41]:
# Split the KFold indexes into separate training and testing datasets 
def KFolds(keypoints_array, label_data_hot, n_splits=5):
    # Initialize KFold
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Initialize lists to store results
    losses = []
    accuracies = []
    
    # Track fold number
    fold_no = 1
    
    # Get the training and testing indexes based on the fold
    for train_index, test_index in kf.split(keypoints_array):
        print(f'Training for fold {fold_no} ...')
        
        # Split the data using the fold indices
        X_train = keypoints_array[train_index]
        y_train = label_data_hot[train_index]
        X_test = keypoints_array[test_index]
        y_test = label_data_hot[test_index]

        # Create and train model for this fold
        model = Sequential()
        model.add(Flatten(input_shape = (17,3)))
        model.add(Dense(units=64, activation='relu'))
        model.add(Dense(units=128, activation='relu'))
        model.add(Dropout(0.2)) 
        model.add(Dense(units=256, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(units=512, activation='relu'))
        model.add(Dropout(0.3)) 
        model.add(Dense(units=1024, activation='relu'))
        model.add(Dropout(0.4))
        model.add(Dense(units=2048, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(units=label_data_hot.shape[1], activation='softmax'))
        
        model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
        
        # Fit the model
        history = model.fit(
            X_train, 
            y_train, 
            epochs=200, 
            batch_size=35, 
            verbose=1
        )
        
        # Evaluate on test data
        loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
        print(f'Fold {fold_no} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')
        
        # Store the metrics for this fold
        losses.append(loss)
        accuracies.append(accuracy)
        
        # Increment fold counter
        fold_no += 1
    
    # Calculate and print average metrics
    avg_loss = sum(losses) / len(losses)
    avg_accuracy = sum(accuracies) / len(accuracies)
    
    print("\n---------- K-Fold Cross Validation Results ----------")
    print(f"Average Loss: {avg_loss:.4f}")
    print(f"Average Accuracy: {avg_accuracy:.4f}")
    print(f"Individual fold accuracies: {[f'{acc:.4f}' for acc in accuracies]}")
    print("----------------------------------------------------")
    
    return losses, accuracies

# Usage
losses, accuracies = KFolds(keypoints_array=keypoints_array, label_data_hot=label_data_hot)

Training for fold 1 ...
Epoch 1/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0446 - loss: 2.2873 
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6035 - loss: 0.9678
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5589 - loss: 0.7925
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4506 - loss: 0.8468
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3965 - loss: 0.7975
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6847 - loss: 0.7527
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5494 - loss: 0.8596
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4602 - loss: 1.1107
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5860 - loss: 0.7398
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5494 - loss: 0.8757
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6401 - loss: 0.7088
Epoch 12/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/ste

In [66]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=4)
reshaped_keypoints_array = np.reshape(keypoints_array, shape=(48,51))
model.fit(reshaped_keypoints_array, labels)

KNeighborsClassifier(n_neighbors=4)

In [73]:
model.predict(np.reshape(keypoints_array[15], (-1, 51)))

array([0])

In [74]:
def load_all_images():
    # Root directory containing all numbered folders
    root_dir = os.path.join(os.getcwd(), 'TestImages', 'ResizedImages')
    
    # Number of folders (0-15)
    num_folders = 16
    
    # Create lists to store all images and their labels
    all_images = []
    all_labels = []
    
    # Loop through each folder
    for folder_num in range(num_folders):
        folder_path = os.path.join(root_dir, str(folder_num))
        
        # Check if folder exists
        if not os.path.exists(folder_path):
            print(f"Warning: Folder {folder_path} does not exist.")
            continue
            
        # Get all image files from the folder
        image_files = [f for f in os.listdir(folder_path)
                    if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
        
        print(f"Loading {len(image_files)} images from class {folder_num}")
        
        # Load each image
        for img_file in image_files:
            img_path = os.path.join(folder_path, img_file)
            try:
                # Open the image
                img = Image.open(img_path)
                
                # Convert to numpy array
                img_array = np.array(img)
                
                # Add to the lists
                all_images.append(img_array)
                all_labels.append(folder_num)  # Use folder number as label
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    
    # Convert lists to numpy arrays
    X = np.array(all_images)
    y = np.array(all_labels)
    
    print(f"Total images loaded: {len(all_images)}")
    print(f"Images shape: {X.shape}")
    print(f"Labels shape: {y.shape}")
    
    return X, y  # Return both images and labels

# Load all images and labels
images, labels = load_all_images()

Loading 24 images from class 0
Loading 36 images from class 1
Loading 12 images from class 2
Loading 36 images from class 3
Loading 12 images from class 4
Loading 24 images from class 5
Loading 24 images from class 6
Loading 24 images from class 7
Loading 36 images from class 8
Loading 12 images from class 9
Loading 36 images from class 10
Loading 12 images from class 11
Loading 24 images from class 12
Loading 24 images from class 13
Loading 36 images from class 14
Loading 12 images from class 15
Total images loaded: 384
Images shape: (384, 1000, 660, 3)
Labels shape: (384,)


In [77]:
def keypoints(images):
  keypoints_with_scores = []

  for image in images:
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

    keypoints_with_scores.append(movenet(input_image))


  return keypoints_with_scores


movenet_keypoints = keypoints(images=images)

movenet_keypoints

[array([[[[0.18560174, 0.499188  , 0.6125121 ],
          [0.1477361 , 0.53244126, 0.7594675 ],
          [0.15092541, 0.46471456, 0.75099343],
          [0.17030618, 0.58432966, 0.77345943],
          [0.17060052, 0.42083392, 0.6046614 ],
          [0.32541397, 0.6742303 , 0.7651241 ],
          [0.3646839 , 0.35140046, 0.8484663 ],
          [0.6197824 , 0.7753371 , 0.9108351 ],
          [0.6692008 , 0.29682276, 0.9077188 ],
          [0.48662984, 0.671418  , 0.8171445 ],
          [0.5092567 , 0.39840382, 0.5932305 ],
          [0.8562911 , 0.63918155, 0.51624453],
          [0.8688652 , 0.42692658, 0.51500905],
          [0.98397845, 0.680927  , 0.08208431],
          [0.97975296, 0.37597036, 0.02569294],
          [1.0575154 , 0.37539726, 0.01129639],
          [1.0082326 , 0.5863816 , 0.01488248]]]], dtype=float32),
 array([[[[0.1565897 , 0.49252456, 0.5859899 ],
          [0.12644143, 0.51460886, 0.39750624],
          [0.1306628 , 0.46311986, 0.8445015 ],
          [0.1292333 

In [100]:
keypoints_array = np.array(movenet_keypoints)
keypoints_array = keypoints_array.squeeze(axis=(1,2))

keypoints_array.shape

(384, 17, 3)

In [79]:
model = KNeighborsClassifier(n_neighbors=4)
reshaped_keypoints_array = np.reshape(keypoints_array, shape=(384,51))
model.fit(reshaped_keypoints_array, labels)

KNeighborsClassifier(n_neighbors=4)

In [99]:
model.predict(np.reshape(keypoints_array[314], (-1, 51)))

array([6])